#CONSOLIDATION TABLES Procedures:

### Configuration:

---



In [ ]:
from google.cloud import bigquery

client = bigquery.Client()
ENV = "dev"             ### REF ENVIRONMENT (DEV/QAS/PRO)  ###
DATASET = "silver"      ### REF LAYER (BRONZE/SILVER/GOLD) ###
PREFIX = "ltm_"         ### LTM Process                    ###
SUFFIX = ""             ### ATENTION !!!!!!!!!!!!!!!!!!!!! ###
NO_DROP = "TRUE"        ### REF TRUE / FALSE               ###

# Company List with original name and new name:
companies = [
    ("MONARCH", "Shape MHS - Monarch"),
    ("CAPITAL", "Shape CHC - Capital"),
    ("FUSION", "Shape TUCSON - Fusion"),
    ("OVER THE MOON", "Shape OTM - Over the Moon - In Phase"),
    ("A1 REFRIGERATION", "Shape AONE - A One"),
    ("LBA", "Shape LBA - LBA"),
    ("PRODIGY PLUMBING", "Shape LBCA - Prodigy Plumbing"),
    ("DEAR", "Shape DEAR - Dear Services"),
    ("HEALTHY HOME", "Shape HHWI - Healthy Home"),
    ("CHAD LOVE", "Shape CLS - Chad Love"),
    ("H AND E Comfort", "Shape HECS - H&E Comfort"),
    ("JSP", "Shape JSP - John Stevenson"),
    ("ICE COOLING", "Shape ICO - ICE"),
    ("COMPLETE COMFORT", "Shape INDY - Complete Comfort"),
    ("SYNERGY HOME", "Shape LEX - Synergy Home"),
    ("HOWZE", "Shape HZE - Howze Plumbing"),
    ("PROSTAR SERVICES", "Shape NCVA - Prostar"),
    ("ABSOLUTE PLUMBING", "Shape AHS - Absolute Plumbing"),
    ("PERSONAL", "Shape PPP - Personal PHC"),
    ("MY GUY", "Shape MGY - My Guy"),
    ("NORTHSTAR", "Shape NS - Northstar Services"),
    ("SUPREME", "Shape SST - Supreme Service"),
    ("SPARTAN", "Shape JFSP - Spartan Plumbing"),
    ("PERFECT TEMP", "Shape PTHC - Perfect Temp"),
    ("PILOT", "Shape PHS - Pilot Plumbing"),
    ("COSMIC COMFORT", "Shape COS - Jantz Cosmic Comfort"),
    ("GREEN ENERGY", "Shape GEM - Green Energy"),
    ("SHARP", "Shape NEWE - Sharp PHC"),
    ("DAFFY DUCTS", "DAFFY DUCTS")
]

### `Drop Tables: Variable Layer (Bronze/Silver/Gold)`

---



In [ ]:
if (NO_DROP == "FALSE"):
    table = f"gp-technologies-{ENV}.{DATASET}.ltm_consolidate"

    sql_statements = [
        f"""
        DROP TABLE IF EXISTS {table};
        """
    ]

    # Run sql_statements en BigQuery
    for sql in sql_statements:
        query_job = client.query(sql)
        query_job.result()

    # Result:
    print(f"DROP TABLE {table}... Successfully.")
else:
      print(f"DROP NOT ACTIVATE.")

DROP NOT ACTIVATE.


### `CLYCLE SCRIPT FOR CONSOLIDATION`
---

In [ ]:
# SCRIPT CONSOLIDATION EN CICLO PARA COMPANIES:
union_queries = []
table = f"gp-technologies-{ENV}.{DATASET}.ltm_consolidate"

# Iterar sobre cada compañía en la lista
for company_name, company_new_name in companies:
    source = f"{PREFIX}{company_name.replace(' ', '_').lower()}"
    table_id = f"gp-technologies-{ENV}.{DATASET}.{source}"

    union_queries.append(
        f"SELECT * FROM {table_id}"
    )

# Unir todas las consultas con UNION ALL
final_sql = f"""
CREATE OR REPLACE TABLE {table} AS
WITH combined_data AS (
    {' UNION ALL '.join(union_queries)}
)
SELECT * FROM combined_data;
"""
#print(final_sql)

# Ejecutar la consulta en BigQuery
query_job = client.query(final_sql)
query_job.result()

destination_table = client.get_table(table)
print(f"Consolidation for Table {destination_table}... Total rows {destination_table.num_rows} Successfully")

Consolidation for Table gp-technologies-dev.silver.ltm_consolidate... Total rows 8640 Successfully


In [ ]:
# SCRIPT CONSOLIDATION EN CICLO PARA COMPANIES VISTA AGRUPADA:
union_queries = []
table = f"gp-technologies-{ENV}.{DATASET}.ltm_consolidate_group"

# Iterar sobre cada compañía en la lista
for company_name, company_new_name in companies:
    source = f"{PREFIX}{company_name.replace(' ', '_').lower()}"
    table_id = f"gp-technologies-{ENV}.{DATASET}.{source}"

    union_queries.append(
        f"SELECT * FROM {table_id}"
    )

# Unir todas las consultas con UNION ALL
final_sql = f"""
CREATE OR REPLACE VIEW {table} AS (
  WITH companies AS (SELECT * FROM gp-technologies-dev.bronze.companies)
SELECT LTM.company_new_name
     , C.company_join_date AS middle_point
     , LTM.month AS month
     , LTM.revenue_total
     , LTM.single_jobs_total
     , LTM.manpower_sj_qty, LTM.manpower_svc_pj_qty, LTM.manpower_repl_pj_qty
     , LTM.sold_hours_total_sold_hrs
     , LTM.sj__usd_100_total
     , LTM.recalls_total
     , LTM.sj_repair_avg_usd_total
     , LTM.conversion_usd_avg_total
     , LTM.sj_mktestimates_total
     , LTM.sj_mkte_conv_usd_total
     , LTM.af_prj_avg_usd_total
     , LTM.equip_avg_usd_total
     , LTM.svc_prj_avg_usd_total
  FROM companies C
  JOIN (
       SELECT company_new_name
            , month
            , SUM(revenue_total) AS revenue_total
            , ROUND(SUM(single_jobs_total),0) AS single_jobs_total
            , ROUND(SUM(manpower_sj_qty),0) AS manpower_sj_qty, ROUND(SUM(manpower_svc_pj_qty),0) AS manpower_svc_pj_qty, ROUND(SUM(manpower_repl_pj_qty),0) AS manpower_repl_pj_qty
            , ROUND(SUM(sold_hours_total_sold_hrs),0) AS sold_hours_total_sold_hrs
            , ROUND(SUM(sj__usd_100_total),0) AS sj__usd_100_total
            , ROUND(SUM(recalls_total),0) AS recalls_total
            , SUM(sj_repair_avg_usd_total) AS sj_repair_avg_usd_total
            , SUM(conversion_usd_avg_total) AS conversion_usd_avg_total
            , ROUND(SUM(sj_mktestimates_total),0) AS sj_mktestimates_total
            , SUM(sj_mkte_conv_usd_total) AS sj_mkte_conv_usd_total
            , SUM(af_prj_avg_usd_total) AS af_prj_avg_usd_total
            , SUM(equip_avg_usd_total) AS equip_avg_usd_total
            , SUM(svc_prj_avg_usd_total) AS svc_prj_avg_usd_total
         FROM gp-technologies-dev.silver.ltm_consolidate
        GROUP BY company_new_name, month
        ORDER BY company_new_name, month
       ) LTM
    ON LTM.company_new_name = C.company_new_name
 ORDER BY middle_point
);
"""
#print(final_sql)

# Ejecutar la consulta en BigQuery
query_job = client.query(final_sql)
query_job.result()

destination_table = client.get_table(table)
print(f"View {destination_table} created... Total rows {destination_table.num_rows} Successfully")

View gp-technologies-dev.silver.ltm_consolidate_group created... Total rows 0 Successfully
